In [ ]:
!pip install  whisperx

In [ ]:
!git lfs install

In [ ]:
!git clone https://huggingface.co/Systran/faster-whisper-large-v3

In [ ]:
!git lfs pull

In [ ]:
!apt-get install libcudnn8 libcudnn8-dev

In [ ]:
import json

with open("config.json", "r") as f:
    config_argument = json.load(f)

audio_file_path = config_argument["audio_file_path"]


print( "audio_file_path: ", audio_file_path )


In [ ]:
import os

result = []

# 多個資料夾路徑
folder_paths = [
    audio_file_path,
    # 可繼續加入其他資料夾路徑
]

for folder_path in folder_paths:
    print(f'\n📂 檔案來自資料夾：{folder_path}')

    # 讀出所有 .wav 檔案
    wav_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.wav')]

    # 根據檔名數字排序
    try:
        wav_files_sorted = sorted(wav_files, key=lambda x: int(os.path.splitext(x)[0]))
    except ValueError:
        print("⚠️ 檔名無法轉為數字，跳過此資料夾")
        continue

    # 印出完整路徑
    for wav_file in wav_files_sorted:
        full_path = os.path.join(folder_path, wav_file)
        print(full_path)
        result.append(full_path)

**基礎ASR**

In [ ]:
import whisperx
import gc
import torch


device = "cuda"
compute_type = "float16"
answer = ""

model = whisperx.load_model("./faster-whisper-large-v3", device, compute_type=compute_type)


for wav_file in result:

  name = wav_file.split("/")[-1].replace(".wav", "")

  audio = whisperx.load_audio( wav_file )
  result = model.transcribe(audio)

  result["segments"][0]["text"] = result["segments"][0]["text"].strip()

  answer = answer + name + "\t" + result["segments"][0]["text"] + "\n"

  print( result["segments"][0]["text"] )

with open("./val_asr_1.txt", "w") as f:
    f.write(answer)

**改進ASR**

In [ ]:
import whisperx
import gc
import torch


device = "cuda"
compute_type = "float16"
answer = ""



for wav_file in result:
    name = wav_file.split("/")[-1].replace(".wav", "")
    audio = whisperx.load_audio(wav_file)
    result = model.transcribe(audio)  # 若確定語言可指定

    text = " ".join([seg["text"].strip() for seg in result["segments"]])

    answer += name + "\t" + text + "\n"

    print(text)

with open("./val_asr_2.txt", "w") as f:
    f.write(answer)


timestaamp index

In [ ]:
import whisperx
import gc
import torch
whisperx.load_model

device = "cuda"
compute_type = "float16"
data_dic = {}

model = whisperx.load_model("./faster-whisper-large-v3", device, compute_type=compute_type)


for wav_file in result:

  print(wav_file)

  name = wav_file.split("/")[-1].replace(".wav", "")

  audio = whisperx.load_audio( wav_file )
  result = model.transcribe(audio)

  result["segments"][0]["text"] = result["segments"][0]["text"].strip()

  # print(result["segments"]) # before alignment
  print( result["segments"][0]["text"] )

  model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
  result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=True)

  print(result["segments"]) # after alignment


  index = 0
  data_list = []
  temp_dic = {}


  for i in result["segments"]:
      for c in i.get("chars", []):  # 安全取出 chars，避免 KeyError

          temp_dic = {
              "index": index,
              "char": c.get("char", ""),  # 預設空字元
              "start": c.get("start"),    # 如果沒有會是 None
              "end": c.get("end")
          }

          data_list.append(temp_dic)
          index += 1

  data_dic[name] = data_list


In [ ]:
import json
with open("./whisperx_char_level.json", "w", encoding="utf-8") as f:
    json.dump(data_dic, f, indent=2, ensure_ascii=False)